<a href="https://www.kaggle.com/code/naciyenur/fruits-and-vegetables-cnn?scriptVersionId=264200844" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import os

count = 0
for dirname, _, filenames in os.walk('/kaggle/input/fruit-and-vegetable-image-recognition'):
    count += len(filenames)
print(f"Toplam dosya sayısı: {count}")


In [ ]:
import os
from collections import Counter

# Dataset klasörü
dataset_path = '/kaggle/input/fruit-and-vegetable-image-recognition'

# Tüm dosya uzantılarını toplama
extensions = []
for dirname, _, filenames in os.walk(dataset_path):
    for filename in filenames:
        ext = os.path.splitext(filename)[1].lower()  # .jpg, .jpeg, .png gibi
        extensions.append(ext)

# Kaç tane her uzantı var?
ext_counts = Counter(extensions)
print("Dosya uzantıları ve sayıları:")
for ext, count in ext_counts.items():
    print(f"{ext}: {count}")


****VERİ ÖN İŞLEME AŞAMASI****

In [ ]:
# Gerekli kütüphanelerin import edilmesi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset_path = '/kaggle/input/fruit-and-vegetable-image-recognition'

In [ ]:
# dosyaları dataframe aktarıyoruz
filepaths = []
labels = []

In [ ]:
for root, _, files in os.walk(dataset_path):
    for file in files:
        filepaths.append(os.path.join(root, file))
        labels.append(os.path.basename(root))  

df = pd.DataFrame({'filepath': filepaths, 'label': labels})

In [ ]:
 df.nunique()

In [ ]:
df["label"].value_counts()
#hangi sınıfta kaç tane görüntü var görüyoruz 

In [ ]:
df.info()
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# veri görselleştirme kütüphaneleri

In [ ]:
plt.figure(figsize=(12,10))
sns.countplot(y='label', data=df, order=df['label'].value_counts().index)
plt.title('Sınıf Dağılımı')
plt.xlabel('Görüntü Sayısı')
plt.ylabel('Sınıf')
plt.show()


HER GÖRSELİN BOYUTU VE FARKINI İNCELEMEMİZ GEREKİYOR

In [ ]:
from PIL import Image
#Image sınıfı, resim dosyalarını açmak, oluşturmak ve üzerinde işlem yapmak için kullanılır.

In [ ]:
widths = []
heights = []
formats = []

In [ ]:
for path in df['filepath'][:1000]: 
    img = Image.open(path)
    widths.append(img.width)
    heights.append(img.height)
    formats.append(img.format)
    #1000 tane örnek alıyoruz ve img nesnesi kullanarak inceliyoruz

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
sns.histplot(widths, kde=True)
plt.title('Görüntü Genişlik Dağılımı')

plt.subplot(1,2,2)
sns.histplot(heights, kde=True)
plt.title('Görüntü Yükseklik Dağılımı')
plt.show()

Her iki histogram da sağa doğru çarpık (skewed to the right) bir dağılım gösteriyor. Bu, veri setindeki görüntülerin çoğunluğunun belirli bir değer aralığında toplandığını, ancak az sayıda görüntünün çok daha büyük genişlik veya yükseklik değerlerine sahip olduğunu ifade eder.

In [ ]:
pd.Series(formats).value_counts()

os.path ile dosya uzantısına bakıyoruz ama Image kütüphanesi görüntüyü açıp gerçek format bilgisini döndürüyor.

In [ ]:
import random
classes = df['label'].unique()
plt.figure(figsize=(15,8))

for i, cls in enumerate(classes[:6]):  
    sample_path = random.choice(df[df['label']==cls]['filepath'].tolist())
    img = Image.open(sample_path)
    
    plt.subplot(2,3,i+1)
    plt.imshow(img)
    plt.title(cls)
    plt.axis('off')

plt.show()

ilk 6 sınıftan rastgele resim çıkıyor ve bu sayede çeşitliliği görmüş oluyoruz.

In [ ]:
#dosya uzunatıları
pd.Series([os.path.splitext(p)[1].lower() for p in df['filepath']]).value_counts()

In [ ]:
aspect_ratios = [w/h for w,h in zip(widths, heights)]


In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(aspect_ratios, bins=30)
plt.title("Aspect Ratio Dağılımı")
plt.show()


Görsellerin çoğu .jpg bazıları ise .jpeg ve .png. Modelimiz için bunları tek formata dönüştürücez. jpeg -> .jpg olarak değiştirebiliriz. .png görsellierini de .jpg'ye PIL ile kolayca dönüştürebiliriz. Böylece ImageDataGenerator veya PyTorch/TF Dataset pipeline’ında hata riskini azaltırız.

In [ ]:
import shutil

output_dir = '/kaggle/working/fruit_veg_dataset'
os.makedirs(output_dir, exist_ok=True)

new_filepaths = []

for idx, row in df.iterrows():
    path = row['filepath']
    ext = os.path.splitext(path)[1].lower()
    
    # Görseli aç ve RGB’ye çevir
    img = Image.open(path).convert('RGB')
    
    # Yeni kaydetme yolu
    new_path = os.path.join(output_dir, f"{idx}.jpg")
    img.save(new_path)
    new_filepaths.append(new_path)

# DataFrame’i güncelle
df['filepath'] = new_filepaths


In [ ]:
for idx, row in df.iterrows():
    path = row['filepath']
    ext = os.path.splitext(path)[1].lower()
    if ext in ['.png', '.jpeg']:
        img = Image.open(path).convert('RGB')
        new_path = os.path.splitext(path)[0] + '.jpg'
        img.save(new_path)
        df.at[idx, 'filepath'] = new_path

In [ ]:
pd.Series([os.path.splitext(p)[1].lower() for p in df['filepath']]).value_counts()

şimdi tüm verilerimiz .jpg oldu. İşlemlerimize devam edebiliriz.

****Train / Validation / Test setlerine ayırma****
Bunun için sklearn kütüphanesini kullanıyoruz.


In [ ]:

train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

# burada stratify kullanrak her sınıfın oranını koruyoruz ki modelimiz doğru sonuçlar versin.

In [ ]:
print(train_df)

In [ ]:
img_size = (128, 128)   # resimleri yeniden boyutlandırma
batch_size = 32

In [ ]:
import tensorflow as tf

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255)
# tüm görselleri 0-255 aralığında 0-1 aralığına ölçeklendirdik. bu işlem modelin daha hızlı ve 
# stabil öğrenmesi için standart bir ön işlmeme adımıdır.

#eğitim verileri
train_gen = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"   # çok sınıflı sınıflandırma
)
#doğrulama verileri
val_gen = datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)
#test verileri
test_gen = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)


train seti 3060 görsel buldu ve toplamda 36 farklı sınıfa dağıttı.
validation setinde 382 görsel bulundu ve toplamda 36 farklı sınıfa dağıttı.
test setinde 383 görsel bulundu ve toplamda 36 farklı sınıfa dağıttı.

In [ ]:
# Artık elimizde 3 tane generator var (train_gen, val_gen, test_gen), bunları oluşturacağmız CNN modeline verebiliriz.

CNN (Convolutional Neural Network) dediğimiz yapı görsellerden otomatik özellik çıkarma işini yapar.
Şu parçaları içerir:

***Conv Katmanı (Convolutional Layer):***

Görsellerde kenar, köşe, renk geçişi gibi özellikleri filtrelerle yakalar.

Örn: layers.Conv2D(32, (3,3), activation='relu')
→ 32 tane 3x3 filtre uygular, aktivasyon fonksiyonu ReLU.

***Pooling Katmanı (MaxPooling):***

Görselin boyutunu küçültür ama en önemli bilgileri korur.

Örn: layers.MaxPooling2D((2,2))
→ 2x2’lik pencereden maksimum değeri alır.

***Dropout Katmanı:***

Overfitting’i önlemek için bazı nöronları rastgele kapatır.

Örn: layers.Dropout(0.5) → %50’sini kapatır.

Flatten + Dense (Fully Connected):

Özellikleri düzleştirip (Flatten) klasik sinir ağına verir.

Dense(128, activation='relu') → gizli katman.

Dense(num_classes, activation='softmax') → çıktı katmanı, sınıfları tahmin eder.


In [ ]:
num_classes = len(train_gen.class_indices)

from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),#görsellerden özellik çıkarımı yapıyor
    layers.MaxPooling2D((2,2)), # Görselleri küçültüp önemli özellikleri koruyor.

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),# flatten tek boyutlu vektöre çeviriyor dense katmanına bağlamk için
    layers.Dropout(0.5), #Modelin aşırı öğrenmesini (overfitting) önlemek için bazı nöronları rastgele kapatıyor. Eğer overfitting görürsek dropout oranını arttırabiliriz.
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # çıktı katmanı
])


In [ ]:
#modeli derleme 
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


modeli eğitme

In [ ]:
history = model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen
)


In [ ]:
# Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

# Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()


modelimiz aşırı öğrenmeye uğradı yani eğitim doğruluğu hızla artıyor ama doğrulama doğruluğu 55-60 civarında takılı kalıyor. Validation loss ise özellikle 5. epoch dan sonra artmaya başlıyor. Modelimiz sıfırdan öğrendiği için düşük bir doğruluk kazandı. Çünkü dataset karmaşık ve çok sınıflı. EfficientNet / MobileNet / ResNet gibi hazır modeller kullanamyı deneyelim.

In [ ]:
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc:.2f}")


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
img_size = (128, 128)
batch_size = 32

In [ ]:
# Train için augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [ ]:
# Validation & Test için sadece normalize
val_test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# DataFrame üzerinden okuma
train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)
val_gen = val_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

test_gen = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

In [ ]:
num_classes = len(train_gen.class_indices)

In [ ]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(128,128,3))
base_model.trainable = False   # önce dondur

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation="softmax")
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# İlk Eğitim (Feature Extraction)
history = model.fit(
    train_gen,
    epochs=15,
    validation_data=val_gen
)

In [ ]:
#fine turning
base_model.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # küçük LR
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen
)


In [ ]:
loss, acc = model.evaluate(test_gen)
print("Test Accuracy:", acc)

***modelimizin doğruluğunu arttırmak için data augmentation'u zenginleştirebiliriz***

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
img_size = (128, 128)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

In [ ]:
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

val_gen = val_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

test_gen = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

In [ ]:
num_classes = len(train_gen.class_indices)

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(128,128,3))
base_model.trainable = False  # İlk etapta dondur

In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    
    layers.Dense(256, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(num_classes, activation="softmax")
])


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
lr_schedule = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)


history = model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen,
    callbacks=[early_stop, lr_schedule]
)

In [ ]:
# Fine-tuning için base_model'i açıyoruz
base_model.trainable = True  

# Yeni optimizer tanımla (önceki optimizer'ı kullanma!)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tuning eğitim (initial_epoch ile kaldığın yerden devam et)
history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,               # toplam epoch
    initial_epoch=history.epoch[-1],  # önceki eğitimden kaldığın yerden devam
    callbacks=[early_stop, lr_schedule]
)


In [ ]:
loss, acc = model.evaluate(test_gen)
print("Test Accuracy:", acc)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2 # %20 validation
)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Pretrained Model: MobileNetV2
base_model = MobileNetV2(input_shape=(128,128,3), include_top=False, weights="imagenet")
base_model.trainable = False  # önce sadece üst katmanları eğitelim

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(train_generator.num_classes, activation="softmax")
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])


early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True)


history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop, checkpoint]
)

In [ ]:
loss, acc = model.evaluate(val_generator)
print("Validation/Test Accuracy:", acc)
print("Validation/Test Loss:", loss)


In [ ]:
# Accuracy & Loss grafikleri
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label="Train Acc")
plt.plot(history.history['val_accuracy'], label="Val Acc")
plt.legend()
plt.title("Accuracy")

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label="Train Loss")
plt.plot(history.history['val_loss'], label="Val Loss")
plt.legend()
plt.title("Loss")
plt.show()

In [ ]:
y_true = val_generator.classes
y_pred = np.argmax(model.predict(val_generator), axis=1)

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=False, cmap="Blues")
plt.title("Confusion Matrix")
plt.show()